In [1]:
# !python train.py

In [1]:
import torch_geometric.datasets as datasets
from gnnexplain.nn.gcn import Model
import torch

cora = datasets.Planetoid(root='data', name='Cora', split='full', transform=None)
model = Model(cora)
checkpoint = torch.load('./checkpoints/cora-epoch=49-v1.ckpt')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [2]:
from gnnexplain.model.nb import OptimizingExplainer

expl = OptimizingExplainer(n_trials=1000, n_cluster_max=35)
expl.fit(cora, model)

/home/apluska/miniconda3/envs/gnnexplain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-03-11 18:06:41,352] A new study created in memory with name: no-name-1ea69ecc-9023-4a62-8af4-cf30301d6b6d
[I 2024-03-11 18:06:44,898] Trial 0 finished with value: 0.356 and parameters: {'n_clusters_0': 9, 'n_clusters_1': 24, 'n_clusters_2': 11, 'n_clusters_3': 27, 'n_clusters_4': 25}. Best is trial 0 with value: 0.356.
[I 2024-03-11 18:06:48,183] Trial 1 finished with value: 0.578 and parameters: {'n_clusters_0': 11, 'n_clusters_1': 34, 'n_clusters_2': 14, 'n_clusters_3': 28, 'n_clusters_4': 2}. Best is trial 1 with value: 0.578.
[I 2024-03-11 18:06:51,828] Trial 2 finished with value: 0.286 and parameters: {'n_clusters_0': 4, 'n_clusters_1': 8, 'n_clusters_2': 30, 'n_clusters_3': 30, 'n_clusters_4': 24}. Be

In [30]:
from gnnexplain.model.nb import Explainer

expl = Explainer(
    n_clusters_0=22,
    n_clusters_1=2,
    n_clusters_2=21,
    n_clusters_3=12,
    n_clusters_4=7
)
expl.fit(cora, model)

In [32]:
expl.accuracy(cora, 'test')

0.733